In [1]:
import pandas as pd
from tqdm.autonotebook import tqdm
import re

/tmp/ipykernel_11210/3235371013.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
df = pd.read_csv("../parsing/paragraphs.csv.zip", index_col=0)
df = df.dropna()
df = df[df["cleaned_text"].apply(lambda x: len(x.split()) > 1)]
df = df.reset_index(drop=True)
df.head()

/tmp/ipykernel_11210/3335727259.py:1: DtypeWarning: Columns (0,1,2,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../parsing/paragraphs.csv.zip", index_col=0)


,company,year,sector,report_type,paragraph,original_text,cleaned_text
0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,1.0,СОЦИАЛЬНЫЙ ОТЧЕТ ММК ЗА 2014 ГОД МАГНИТОГОРСКИ...,социальный отчёт ммк за год магнитогорский мет...
1,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,4.0,Обращение председателя совета директоров.........,обращение председатель совет директор параметр...
2,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,6.0,Обращение председателя совета директоров,обращение председатель совет директор
3,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,7.0,и сервисных услуг. Мы практически завершили дл...,сервисный услуга мы практически завершить длит...
4,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,9.0,"Виктор ФИЛИПпОВИЧ Рашников, председатель совет...",виктор филиппович рашник председатель совет ди...


In [3]:
topics = pd.read_csv("topics.csv.zip")
topics.head()

,Экология,Климат,Энергия,Воздух,Вода,Отходы,Биоразнообразие,Социально-культурное воздействи,max_score,max_topic
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,0.260378,0.160128,0.0,0.0,0.0,0.0,0.0,0.0,0.260378,Экология
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,0.130189,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.130189,Экология
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [4]:
topics["max_topic"].value_counts()

Экология           285751
Климат             208173
Энергия             78585
Отходы              59408
Вода                41847
Биоразнообразие      8287
Воздух               6894
Name: max_topic, dtype: int64

In [5]:
df.shape, topics.shape

((1963163, 7), (1963163, 10))

In [6]:
df = df.join(topics)
df.head()

,company,year,sector,report_type,paragraph,original_text,cleaned_text,Экология,Климат,Энергия,Воздух,Вода,Отходы,Биоразнообразие,Социально-культурное воздействи,max_score,max_topic
0,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,1.0,СОЦИАЛЬНЫЙ ОТЧЕТ ММК ЗА 2014 ГОД МАГНИТОГОРСКИ...,социальный отчёт ммк за год магнитогорский мет...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,4.0,Обращение председателя совета директоров.........,обращение председатель совет директор параметр...,0.260378,0.160128,0.0,0.0,0.0,0.0,0.0,0.0,0.260378,Экология
2,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,6.0,Обращение председателя совета директоров,обращение председатель совет директор,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,7.0,и сервисных услуг. Мы практически завершили дл...,сервисный услуга мы практически завершить длит...,0.130189,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.130189,Экология
4,ПАО «ММК»,2014,Металлургическая и горнодобывающая,СО,9.0,"Виктор ФИЛИПпОВИЧ Рашников, председатель совет...",виктор филиппович рашник председатель совет ди...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [7]:
grouped = df.groupby(["company", "year", "max_topic"])["cleaned_text"].agg(' '.join)
grouped.head()

company      year        max_topic      
Abbott       2013, 2012  Биоразнообразие    недостаточность питание повышенный потребность...
                         Климат             программа проводиться июнь декабрь год направи...
                         Отходы             о компания это один крупный многопрофильный ко...
                         Экология           молекулярный диагностика детский лечебный пита...
Alcoa, Inc.  2004        Воздух                        окисел азот в тысяча метрический тонна
Name: cleaned_text, dtype: object

In [8]:
from collections import defaultdict

sheets_dict = {}
base_dict = {}
re_dict = {}
len_topics = {}

excel = pd.ExcelFile("./esg_topics.xlsx")
for sheet_name in excel.sheet_names:
    sheet_df = pd.read_excel(excel, sheet_name=sheet_name, header=None)
    sheets_dict[sheet_name] = {row[0]: row[1] for _, row in sheet_df.iterrows()}
    base_dict[sheet_name] = {row[0]: 0 for _, row in sheet_df.iterrows()}
    re_dict[sheet_name] = re.compile("(" + "(?!\w)|".join(sheet_df[0].tolist()) + "(?!\w))")
    len_topics[sheet_name] = defaultdict(int)
    for val in sheets_dict[sheet_name].values():
        len_topics[sheet_name][val] += 1

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [9]:
def get_score(words: dict[str, int], topic: str) -> int:
    count_words = {key: 0 for key in len_topics[topic]}
    for word in words:
        count_words[sheets_dict[topic][word]] += 1
    base_percent = count_words[1] / len_topics[topic][1]
    advanced_percent =  count_words[2] / len_topics[topic][2] if count_words.get(2, False) else 0
    if base_percent < 0.20:
        return 1
    elif base_percent < 0.40:
        return 2
    elif base_percent < 0.60:
        return 3
    elif base_percent < 0.80:
        return 4
    elif base_percent <= 1 and advanced_percent == 0:
        return 5
    elif base_percent <= 1 and advanced_percent < 0.40:
        return 6
    elif base_percent <= 1 and advanced_percent < 0.80:
        return 7
    elif base_percent <= 1 and advanced_percent <= 1:
        return 7
    else:
        return 0

In [10]:
import copy


scores = []
percents = []
for (company, year, topic), text in tqdm(grouped.items(), total=grouped.shape[0]):
    paragraph_dict = copy.deepcopy(base_dict)[topic]
    words = re_dict[topic].findall(text)
    for word in words:
        paragraph_dict[word] = 1
    scores.append(get_score(paragraph_dict, topic))

  0%|          | 0/7640 [00:00<?, ?it/s]

In [11]:
df.to_csv("paragraphs.csv.zip", index=False)

In [12]:
grouped = pd.DataFrame(grouped) 
grouped["score"] = scores
grouped.head()

cleaned_text  \
company     year       max_topic                                                            
Abbott      2013, 2012 Биоразнообразие  недостаточность питание повышенный потребность...   
                       Климат           программа проводиться июнь декабрь год направи...   
                       Отходы           о компания это один крупный многопрофильный ко...   
                       Экология         молекулярный диагностика детский лечебный пита...   
Alcoa, Inc. 2004       Воздух                      окисел азот в тысяча метрический тонна   
...                                                                                   ...   
Яндекс      2021       Воздух           вводный пересчёт гдж теплота сгорание один лит...   
                       Климат           углеродный след в год впервые представить пове...   
                       Отходы           упаковка отход июль приобрести универсальный б...   
                       Экология         информационный безопасность защита персональны...   
                       Энергия          энергоэффективность собственный инфраструктура...   

                                        score  
company     year       max_topic               
Abbott      2013, 2012 Биоразнообразие      5  
                       Климат               7  
                       Отходы               7  
                       Экология             7  
Alcoa, Inc. 2004       Воздух               5  
...                                       ...  
Яндекс      2021       Воздух               5  
                       Климат               7  
                       Отходы               7  
                       Экология             7  
                       Энергия              7  

[7640 rows x 2 columns]

In [14]:
grouped.to_csv("grouped_paragraphs.csv.zip", index=False)